# La migration des érables à sucre au Canada

Ce document est divisé en quatre sections: \
A. Lecture des données \
B. Création de la table de données de travail \
C. Visualisation interactive des données \
D. Réponses aux questions de recherche

In [ ]:
import geopandas
import rasterio
from pathlib import Path
import numpy as np
import pandas as pd
from rasterstats import zonal_stats
import matplotlib.pyplot as plt
from ipywidgets import interact
import matplotlib as mpl

from utils import read_raster, reproject_raster, compute_area, zonal_stats_for_value

%load_ext autoreload
%autoreload 2

## Définition des variables 

In [ ]:
DATA_PATH = Path("data/")
CRS = "EPSG:4326"
TOP_N = 20

# A. Lecture des données 

### Données d'exploitations acéricoles et d'entailles d'érables

Les données d'exploitations et d'entailles proviennent de Statistique Canada. On a télécharché les données pour [2011, 2016](https://www150.statcan.gc.ca/t1/tbl1/fr/cv!recreate.action?pid=3210042301&selectedNodeIds=&checkedLevels=0D1,0D2,0D3,0D4,0D5,1D1&refPeriods=20110101,20160101&dimensionLayouts=layout3,layout3,layout2&vectorDisplay=false) et [2021](https://www150.statcan.gc.ca/t1/tbl1/fr/cv.action?pid=3210036201), par région agricole de recensement (RAR). 

In [ ]:
df_farms = pd.concat(
    [
        pd.read_csv(DATA_PATH / "fermes" / "3210036201_donneesselectionnees.csv", delimiter=";"), 
        pd.read_csv(DATA_PATH / "fermes" / "3210042301_donneesselectionnees.csv", delimiter=";")
    ]
).drop(
    [
        "UNITÉ DE MESURE", 
        "IDENTIFICATEUR D'UNITÉ DE MESURE", 
        "FACTEUR SCALAIRE", 
        "VECTEUR", 
        "COORDONNÉES", 
        "IDENTIFICATEUR SCALAIRE", 
        "STATUS", 
        "SYMBOLE", 
        "TERMINÉ", 
        "DÉCIMALES"
    ], 
    axis=1
).reset_index()

df_farms["GÉO"] = df_farms.apply(
    lambda row: row["GÉO"].split(" [")[0], 
    axis=1
)

df_farms["DGUID"] = df_farms.apply(
    lambda row: row["DGUID"][4:], 
    axis=1
)


df_farms = df_farms.replace(
    "Nombre d'exploitations déclarantes", 
    "num_exploitations"
).replace(
    "Nombre d'entailles", 
    "num_entailles"
).replace(
    "Nombre de fermes déclarantes", 
    "num_exploitations"
).pivot(
    index=["GÉO", "DGUID"], 
    columns=["PÉRIODE DE RÉFÉRENCE", "Entailles d'érables"], 
    values="VALEUR"
)

df_farms.columns = [f"{x}_{y}" for x, y in df_farms.columns.to_flat_index()]

df_farms.head()

### Fichiers des limites géographiques

On téléchargement également les fichiers des [limites géographiques des régions agricoles de recensement (RAR)](https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/index2021-fra.cfm?year=21) à partir du site de Statistique Canada. 

In [ ]:
df_boundaries = geopandas.read_file(
    DATA_PATH / "boundaries" / "lcar000b21a_e.shp"
).to_crs(
    CRS
).drop(
    ["CARUID", "CARENAME", "LANDAREA"], 
    axis=1
).rename(
    columns={"CARFNAME": "area_name"}
)

df_boundaries["DGUID"] = df_boundaries.apply(
    lambda row: row["DGUID"][4:], 
    axis=1
)

df_boundaries["total_area"] = compute_area(df_boundaries)

df_boundaries.head()

On visualise les limites géographiques.

In [ ]:
df_boundaries.plot()

### Carte de la biomasse (2001)

On télécharge ensuite les cartes de la biomasse aérienne de l'érable à sucre en tonnes/ha. Il s'agit d'un fichier de type raster avec une résolution de 250m. 

_Source: Beaudoin, A., Bernier, P. Y., Guindon, L., Villemaire, P., Guo, X. J., Stinson, G., . . . Hall, R. J. (2014). Mapping attributes of Canada’s forests at moderate resolution through kNN and MODIS imagery. Canadian journal of forest research, 44(5), 521-532. [doi:10.1139/cjfr-2013-0401](doi:10.1139/cjfr-2013-0401)_

In [ ]:
filename = DATA_PATH / "cartography" / "ACESAC_volume_250m_final-100.tif"    
biomass_raster, affine, no_data_value = read_raster(filename, crs=CRS)
    
plt.imshow(biomass_raster)
plt.show()

On calcule la biomasse totale (`total_biomass`) pour chacune des régions agricoles de recensement. 

In [ ]:
df_boundaries["stats"] = zonal_stats(
    vectors=df_boundaries['geometry'], 
    raster=biomass_raster, 
    affine=affine, 
    stats="sum", 
    nodata=no_data_value
)

df_boundaries["total_biomass"] = df_boundaries.apply(
    lambda row: row["stats"]["sum"], 
    axis=1
)

df_boundaries = df_boundaries.drop(["stats"], axis=1)

df_boundaries.head()

### Projections de l'enveloppe climatique propice à l'érable à sucre 

On télécharge ensuite les fichiers de projections climatiques pour l'érable à sucre (selon le scénario SDM ANUCLIM et le RCP 8.5) pour les horizons 2011-2040, 2041-2070 et 2071 à 2100. 

On considère une enveloppe "marginale" et une enveloppe "core", selon les définitions suivantes:
- "marginale": enveloppe climatique pour une période définie calibrée selon le min/max des conditions climatiques observées durant la période de référence.
- "core": Enveloppe climatique selon le 5e à 95e centile des conditions climatiques observées durant la période de référence 

Source: https://protect-eu.mimecast.com/s/G3lTCVNkkFqzB6DHGqvem?domain=planthardiness.gc.ca ET McKENNEY, D. W., Pedlar, J. H., Rood, R. B., & Price, D. (2011). Revisiting projected shifts in the climate envelopes of North American trees using updated general circulation models. Global Change Biology, 17(8), 2720-2730.  

#### Projection 2011-2040

In [ ]:
filename = DATA_PATH / "rasters" / "arcp8510000532011-2040.asc"

raster_2011_2040, affine, no_data_value = read_raster(filename, crs=CRS)
    
cmap = mpl.colors.ListedColormap(['black', 'yellow', 'orange'])
bounds = [no_data_value, 1, 3, 4]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

plt.imshow(raster_2011_2040, cmap=cmap, norm=norm)
plt.show()

Pour chacune des régions agricoles de recensement, on calcule le nombre de pixel pour chacune des catégories ("marginale", "core" et "autres").

In [ ]:
period = "2011-2040"

for value, name in [
    (1, "marginal"), 
    (3, "core"), 
    (no_data_value, "rest")
]:
    df_boundaries[f"{name}_{period}"] = zonal_stats_for_value(
        raster=raster_2011_2040, 
        vectors=df_boundaries["geometry"], 
        value=value, 
        stats="count", 
        data_value=100, 
        no_data_value=no_data_value, 
        affine=affine
    )

#### Projection 2041-2070

In [ ]:
filename = DATA_PATH / "rasters" / "arcp8510000532041-2070.asc"

raster_2041_2070, affine, no_data_value = read_raster(filename, crs=CRS)
    
plt.imshow(raster_2041_2070, cmap=cmap, norm=norm)
plt.show()

Pour chacune des régions agricoles de recensement, on calcule le nombre de pixel pour chacune des catégories ("marginale", "core" et "autres").

In [ ]:
period = "2041-2070"

for value, name in [
    (1, "marginal"), 
    (3, "core"), 
    (no_data_value, "rest")
]:
    df_boundaries[f"{name}_{period}"] = zonal_stats_for_value(
        raster=raster_2041_2070, 
        vectors=df_boundaries["geometry"], 
        value=value, 
        stats="count", 
        data_value=100, 
        no_data_value=no_data_value, 
        affine=affine
    )

#### Projection 2071-2100

In [ ]:
filename = DATA_PATH / "rasters" / "arcp8510000532071-2100.asc"
raster_2071_2100, affine, no_data_value = read_raster(filename, crs=CRS)
    
plt.imshow(raster_2071_2100, cmap=cmap, norm=norm)
plt.show()

Pour chacune des régions agricoles de recensement, on calcule le nombre de pixel pour chacune des catégories ("marginale", "core" et "autres").

In [ ]:
period = "2071-2100"

for value, name in [
    (1, "marginal"), 
    (3, "core"), 
    (no_data_value, "rest")
]:
    df_boundaries[f"{name}_{period}"] = zonal_stats_for_value(
        raster=raster_2071_2100, 
        vectors=df_boundaries["geometry"], 
        value=value, 
        stats="count", 
        data_value=100, 
        no_data_value=no_data_value, 
        affine=affine
    )

# B. Création de la table de données de travail

On joint les données téléchargées dans un dataframe comprenant toutes les informations requises pour répondre aux questions de recherche. On calcule quelques statistiques préalables et on nettoie la table de données finale.

In [ ]:
df = pd.merge(
    df_boundaries, 
    df_farms, 
    on="DGUID"
)

pr_df = pd.DataFrame({
    "PRUID": ["10", "11", "12", "13", "24", "35", "46", "47", "48", "59", "60", "61", "62"], 
    "province": [
        "Terre-Neuve-et-Labrador", 
        "Île-du-Prince-Édouard", 
        "Nouvelle-Écosse",
        "Nouveau-Brunswick",
        "Québec",
        "Ontario",
        "Manitoba",
        "Saskatchewan",
        "Alberta",
        "Colombie-Britannique",
        "Yukon",
        "Territoires du Nord-Ouest",
        "Nunavut"
    ]
}
)

df = df.merge(
    pr_df, 
    on="PRUID"
).drop("PRUID", axis=1)

df["rank_biomass"] = df["total_biomass"].rank(method="min", na_option="bottom", ascending=False)
df["rank_entailles"] = df["2021_num_entailles"].rank(method="min", na_option="bottom", ascending=False)
df["rank_exploitations"] = df["2021_num_exploitations"].rank(method="min", na_option="bottom", ascending=False)

df = df.rename({
    "all_2011-2040": "rest_2011-2040", 
    "all_2041-2070": "rest_2041-2070", 
    "all_2071-2100": "rest_2071-2100", 
}, axis=1)

df.head()

# C. Visualisation interactive des données 

On peut visualiser les différentes variables disponibles dans cette carte interactive. 

Attention! Comme il s'agit de données relativement lourdes, on enregistre un délai entre le changement de variable et le rafraichissement de la carte interactive. 

In [ ]:
def _plot(province, column_name):
    return df[df["province"] == province].explore(column_name)
    
interact(
    _plot,
    province= df["province"].unique(),
    column_name=df.columns.tolist()[3:-2]
)

# D. Réponses aux questions de recherche

Pour répondre aux questions de recherche, on définie deux scénarios: 
- Scénario A: On ne considère que les zones vec les conditions climatiques cores.
- Scénario B: On considère les zones avec les conditions climatiques cores et marginales.

On doit également décliner la réponse par rapport aux projections de 2041-2070, puis aux projections de 2071-2100. 

On s'attend donc logiquement à ce que les différences les plus marquantes soient observées selon le scénario A, et les projections de 2071-2100. 

Les conclusions aux questions de recherche sont affichées <span style="color:red"> en rouge </span>à la suite des questions. 

## 1. Où se situent les zones propices au développement de l’érable à sucre au Canada et comment seront-elles affectées par le réchauffement climatique?

### Y a-t-il des zones actuellement propices qui ne le seront plus du tout?

On calcule la perte de zone propice pour les scénarios A et B, ainsi que pour les deux préiodes de projections. 

In [ ]:
# Scénario A 
df["diff_A_2011_vs_2041"] = df["core_2011-2040"] - df["core_2041-2070"]
df["diff_A_2011_vs_2071"] = df["core_2011-2040"] - df["core_2071-2100"]

# Scénario B
df["core_marginal_2011-2040"] = df["core_2011-2040"] + df["marginal_2011-2040"]
df["core_marginal_2041-2070"] = df["core_2041-2070"] + df["marginal_2041-2070"]
df["core_marginal_2071-2100"] = df["core_2071-2100"] + df["marginal_2071-2100"]
df["diff_B_2011_vs_2041"] = df["core_marginal_2011-2040"] - df["core_marginal_2041-2070"]
df["diff_B_2011_vs_2071"] = df["core_marginal_2011-2040"] - df["core_marginal_2071-2100"]

#### Scénario A (2011-2040 vs 2041-2070)

On affiche les données ordonnées en fonction de la perte de zones propices (`diff_A_2011_vs_2041`). `core_2011-2040` et `core_2041-2070` représentent le nombre absolu de pixel à l'intérieur des zones propices, et `rank_biomass` représente le rang de la région agricole de recensement en fonction de la biomasse totale d'érable à sucre. 

À noter qu'on considère la biomasse totale, indépendemment de l'aire totale d'une région de recensement. 

<span style="color:red"> 
On constate que les régions du Nord, du Sud et de l'Est de l'Ontario sont les régions ou on enregistre les plus importantes pertes de territoires. Malgré tout, les projections de 2041-2070 indiquent qu'une grande partie de ces régions seront encore propices au développement de l'érable à sucre. Il en va de même pour la Région agricole de recensement 2 de Terre-Neuve-et-Labrador.
    
À l'inverse, les Région agricole de recensement 2 et 3 du Nouveau-Brunswick, ainsi que la Montérégie (Québec) enregistre également d'importantes pertes, et on constate que ces régions ne seront plus propices au développement de l'érable à sucre en 2041. 
    
De même, ls Région agricole de recensement 3 de Nouvelle-Écosse ainsi que les Régions agricole de recensement 2 et 3 de l'Île-du-Prince-Édouard sont actuellement propices, mais ne sont plus du tout en 2041. 
</span>

In [ ]:
df.sort_values(
    "diff_A_2011_vs_2041", 
    ascending=False
)[[
    "area_name", 
    "province", 
    "diff_A_2011_vs_2041", 
    "core_2011-2040",
    "core_2041-2070", 
    "rank_biomass"
]].reset_index().head(n=TOP_N)

#### Scénario A (2011-2040 vs 2071-2100)

On affiche les données ordonnées en fonction de la perte de zones propices (`diff_A_2011_vs_2071`). `core_2011-2040` et `core_2071-2100` représentent le nombre absolu de pixel à l'intérieur des zones propices, et `rank_biomass` représente le rang de la région agricole de recensement en fonction de la biomasse totale d'érable à sucre. 

<span style="color:red"> 
Les conclusions sont d'autant plus imposantes en 2071. Presque toutes les régions enregistrant une perte ne seront plus propices en 2071. On note quelques exceptions, notament la Mauricie (Québec) et de la Région agricole de recensement 2 de Terre-Neuve-et-Labrabor,  ou une très faible partie du territoire demeure propice au développement de l'érable à sucre. 
</span>

In [ ]:
df.sort_values(
    "diff_A_2011_vs_2071", 
    ascending=False
).reset_index()[[
    "area_name",  
    "province",
    "diff_A_2011_vs_2071", 
    "core_2011-2040",
    "core_2071-2100",
    "rank_biomass"
]].head(n=TOP_N)

#### Scénario B (2011-2040 vs 2041-2070)

On affiche les données ordonnées en fonction de la perte de zones propices (`diff_B_2011_vs_2041`). `core_2011-2040` et `core_2041-2070` représentent le nombre absolu de pixel à l'intérieur des zones propices, et `rank_biomass` représente le rang de la région agricole de recensement en fonction de la biomasse totale d'érable à sucre. 

<span style="color:red"> 
Lorsqu'on considère les conditions climatiques cores et marginales, on enregistre très peu de recul en 2041, à l'exception de la Région agricole de recensement d'Alberta, qui conserve malgré tout une part importante de territoire propice au développement. L'érable à sucre n'est toutefois pas présent dans cette région. 
</span>

In [ ]:
df.sort_values(
    "diff_B_2011_vs_2041", 
    ascending=False
).reset_index()[[
    "area_name",  
    "province",
    "diff_B_2011_vs_2041", 
    "core_marginal_2011-2040", 
    "core_marginal_2041-2070",
    "rank_biomass"
]].head(n=TOP_N)

#### Scénario B (2011-2040 vs 2071-2100)

On affiche les données ordonnées en fonction de la perte de zones propices (`diff_B_2011_vs_2071`). `core_2011-2040` et `core_2071-2100` représentent le nombre absolu de pixel à l'intérieur des zones propices, et `rank_biomass` représente le rang de la région agricole de recensement en fonction de la biomasse totale d'érable à sucre. 


<span style="color:red"> 
En 2071, on constate toutefois qu'une très faible portion de certaines régions seront propice au développement de l'érable à sucre. C'est le cas entre autres de la Montérégie (Québec), des Régions agricole de recensement 1 et 2	(Manitoba) et Vancouver Island-Coast (Colombie-Britannique). 
    
À noter que si ces régions sont considérés comme ayant des conditions climatiques propices, l'érable à sucre n'est présent qu'en Montérégie. 
</span>

In [ ]:
df.sort_values(
    "diff_B_2011_vs_2071", 
    ascending=False
).reset_index()[[
    "area_name",  
    "province",
    "diff_B_2011_vs_2071", 
    "core_marginal_2011-2040", 
    "core_marginal_2071-2100",
    "rank_biomass"
]].head(n=TOP_N)

### Quelle proportion du Canada et des provinces est actuellement propice au développement de l’érable à sucre? Qu'elle deviendra cette proportion avec le réchauffement climatique?

#### Canada 

On calcule la proportion du Canada qui est propice au développement de l'érable à sucre en 2011, 2041 et 2071, selon les scénarios A et B. 

<span style="color:red"> 
On constate que la proportion du territoire canadien propice au développement de l'érable à sucre s'accroit avec le temps, qu'on se fit au scénario A ou B. En effet, l'enveloppe climatique ne rétrécit pas, mais elle se déplace vers le Nord. 
</span>

In [ ]:
sum_A_2011 = df['core_2011-2040'].sum() / (df['rest_2011-2040'].sum() + df['core_marginal_2011-2040'].sum())
sum_B_2011 = df['core_marginal_2011-2040'].sum() / (df['rest_2011-2040'].sum() + df['core_marginal_2011-2040'].sum())

sum_A_2041 = df['core_2041-2070'].sum() / (df['rest_2041-2070'].sum() + df['core_marginal_2041-2070'].sum())
sum_B_2041 = df['core_marginal_2041-2070'].sum() / (df['rest_2041-2070'].sum() + df['core_marginal_2041-2070'].sum())

sum_A_2071 = df['core_2071-2100'].sum() / (df['rest_2071-2100'].sum() + df['core_marginal_2071-2100'].sum())
sum_B_2071 = df['core_marginal_2071-2100'].sum() / (df['rest_2071-2100'].sum() + df['core_marginal_2071-2100'].sum())

print(f"Scénario A: {round(sum_A_2011, 3)} (2011), {round(sum_A_2041, 3)} (2041), {round(sum_A_2071, 3)} (2071)")
print(f"Scénario B: {round(sum_B_2011, 3)} (2011), {round(sum_B_2041, 3)} (2041), {round(sum_B_2071, 3)} (2071)")

#### Provinces

On calcule ensuite ces mêmes proportions à l'échelle des provinces. `sum_X_YYYY` représente la proportion pour l'année de départ d'une projection donnée (`YYYY`) et un scénario (`X`).

<span style="color:red"> 
On observe un recul important de la proportion des zones propices dans les provinces du Nouveau-Brunswick, de l'Ontario et de l'Île-du-Prince-Édouard. En 2071, l'entièreté de ces trois provinces ne sera pas propice au développement de l'érable à sucre (selon le scénario A). La proportion de zones propices demeure plus stable dans le temps considérant le scénario B. 

Au Québec, une plus grande portion de la province serait propice au développement de l'arbre à sucre avec le temps, tant selon le scénario A que le scénario B. Il en va de même pour Terre-Neure-et-Labrador, province qui n'est présentement pas très propice au développement de l'érable à sucre. 
</span>

In [ ]:
df_pr = df.groupby(
    "province"
).sum()

df_pr["sum_A_2011"] = df_pr['core_2011-2040'] / (df_pr['rest_2011-2040'] + df_pr['core_marginal_2011-2040'])
df_pr["sum_B_2011"] = df_pr['core_marginal_2011-2040'] / (df_pr['rest_2011-2040'] + df_pr['core_marginal_2011-2040'])

df_pr["sum_A_2041"] = df_pr['core_2041-2070'] / (df_pr['rest_2041-2070'] + df_pr['core_marginal_2041-2070'])
df_pr["sum_B_2041"] = df_pr['core_marginal_2041-2070'] / (df_pr['rest_2041-2070'] + df_pr['core_marginal_2041-2070'])

df_pr["sum_A_2071"] = df_pr['core_2071-2100'] / (df_pr['rest_2071-2100'] + df_pr['core_marginal_2071-2100'])
df_pr["sum_B_2071"] = df_pr['core_marginal_2071-2100'] / (df_pr['rest_2071-2100'] + df_pr['core_marginal_2071-2100'])

df_pr[[
    "sum_A_2011", 
    "sum_A_2041", 
    "sum_A_2071", 
    "sum_B_2011", 
    "sum_B_2041", 
    "sum_B_2071", 
]]

## 2. Où se concentre la production de sirop d’érable au Canada. Et comment a-t-elle évolué au fil des années?

### Où se concentre la production de sirop d’érable au Canada?

On affiche les données ordonnées en fonction du nombre d'entailles en 2021 (`2021_num_entailles`). On affiche également le nombre d'exploitations en 2021 (`2021_num_exploitations`), et le rang de la région agricole de recensement en fonction de la biomasse totale	(`rank_biomass`), du nombre d'entailles (`rank_entailles`) et du nombre d'exploitations acéricoles (`rank_exploitations`).

<span style="color:red"> 
Le Québec domine l'exploitation des érables à sucre. On retrouve également une importante production au Nouveau-Brunswick, en Ontario et, dans une moindre mesure, en Nouvelle-Écosse. 
</span>

In [ ]:
df.sort_values(
    "2021_num_entailles", 
    ascending=False
).reset_index()[[
    "area_name", 
    "province", 
    "2021_num_entailles", 
    "2021_num_exploitations", 
    "rank_biomass", 
    "rank_entailles",
    "rank_exploitations"
]].head(n=TOP_N)

### Quelles régions ont connu la plus forte croissance au fil des ans?

On calcule ici les différences de nombre d'entailles et entre 2021 et 2011 (`diff_num_entailles`). 

<span style="color:red"> 
Si le Manitoba a connu la plus forte hausse relative, le nombre absolu d'entailles demeure extrêmement bas. Le Nouveau-Brunswick, l'Ontario et la Nouvelle-Écosse ont connu des croissances plus marquées que les régions du Québec. 
</span>

In [ ]:
df["diff_num_entailles"] = (df["2021_num_entailles"] - df["2011_num_entailles"]) / df["2021_num_entailles"]

df.sort_values(
    "diff_num_entailles", 
    ascending=False
).reset_index()[[
    "area_name", 
    "province", 
    "diff_num_entailles",
    "2021_num_entailles", 
    "2011_num_entailles", 
    "2021_num_exploitations",
    "2011_num_exploitations", 
    "rank_biomass",
    "rank_entailles",
    "rank_exploitations"
]].head(n=TOP_N)

## 3. Croiser les données sur la répartition et l'abondance de l’érable à sucre du Canada, et les projections de niche climatique d’ici 2100, avec les données sur les entailles d’étables du recensement de l’agriculture 2021. 

### Quelle proportion (et quelles régions) des entailles actuelles se retrouveront dans des zones qui ne sont plus propices à la croissance de l’érable à sucre?


#### Scénario A (2041-2070)

On affiche les régions agricoles de recensement qui ne se retrouvent plus dans la zone propice. 

<span style="color:red"> 
Le climat des régions agricoles de recensement 2 et 3 de la Nouvelle-Écosse ne sera plus propice au développement de l'érable à sucre dès 2041. Cette région compte près de 400000 entailles et 86 exploitations acéricoles en 2021. 
    
On dénombre également 16 exploitations avec moisn de 3000 entailles dans les régions agricoles de recensement 2, 5 et 7 qui ne se trouveront plus dans un climat propice. 
</span>

In [ ]:
# Scénario A (2041)
df["2041_A_is_propice"] = df["core_2041-2070"] > 0
is_not_propice_2041_A_df = df[df["2041_A_is_propice"] == False].sort_values(
    "2021_num_entailles", 
    ascending=False
).reset_index()

is_not_propice_2041_A_df[[
    "area_name", 
    "province", 
    "2021_num_entailles", 
    "2021_num_exploitations", 
    "rank_entailles",
    "rank_exploitations"
]]

#### Scénario A (2071-2100)

On affiche les régions agricoles de recensement qui ne se retrouvent plus dans la zone propice.

<span style="color:red"> 
À partir de 2071, huit des dix régions produisant le plus grand nombre d'entailles en 2021 ne seront plus du tout propice au développement de l'érable à sucre. On note Chaudière-Appalaches, l'Estrie, le Bas-Saint-Laurent le Centre-du-Québec et la Montérégie (Québec) ainsi que les Régions agricoles de recensement 1 et 4 au Nouveau-Brunswick et laRégion de l'Ouest de l'Ontario. 
</span>

In [ ]:
# Scénario A (2071)
df["2071_A_is_propice"] = df["core_2071-2100"] > 0
is_not_propice_2071_A_df = df[df["2071_A_is_propice"] == False].sort_values(
    "2021_num_entailles", 
    ascending=False
).reset_index()

is_not_propice_2071_A_df[[
    "area_name", 
    "province", 
    "2021_num_entailles", 
    "2021_num_exploitations", 
    "rank_entailles",
    "rank_exploitations"
]]

#### Scénario B (2041-2070)

On affiche les régions agricoles de recensement qui ne se retrouvent plus dans la zone propice. 

<span style="color:red"> 
Selon le scénario B, aucune région importante au développement des produits de l'érable se trouvera totalement exclue de la zone propice en 2041. 
</span>

In [ ]:
# Scénario B (2041)
df["2041_B_is_propice"] = df["core_marginal_2041-2070"] > 0
is_not_propice_2041_B_df = df[df["2041_B_is_propice"] == False].sort_values(
    "2021_num_entailles", 
    ascending=False
).reset_index()

is_not_propice_2041_B_df[[
    "area_name", 
    "province", 
    "2021_num_entailles", 
    "2021_num_exploitations", 
    "rank_entailles",
    "rank_exploitations"
]]

#### Scénario B (2071-2100)

On affiche les régions agricoles de recensement qui ne se retrouvent plus dans la zone propice. 

<span style="color:red"> 
Il en va de même en 2071 également. 
</span>

In [ ]:
# Scénario B (2071)
df["2071_B_is_propice"] = df["core_marginal_2071-2100"] > 0
is_not_propice_2071_B_df = df[df["2071_B_is_propice"] == False].sort_values(
    "2021_num_entailles", 
    ascending=False
).reset_index()

is_not_propice_2071_B_df[[
    "area_name", 
    "province", 
    "2021_num_entailles", 
    "2021_num_exploitations", 
    "rank_entailles",
    "rank_exploitations"
]]

#### Proportions d'entailles et d'exploitations ne se retrouvant plus dans les zones propices selon les scénarios A et B et pour les deux périodes de projection. 

<span style="color:red"> 
Au total, selon le scénario A, c'est 90.9% des entailles et 89.4% des exploitations qui se retrouveront exclues des zones propices en 2071. 
    
Les proportions sont minimes pour le scénario A en 2041, ainsi que pour le scénario B. 
</span>

In [ ]:
ratio_entailles_A_2041 = is_not_propice_2041_A_df["2021_num_entailles"].sum() / df["2021_num_entailles"].sum()
ratio_entailles_A_2071 = is_not_propice_2071_A_df["2021_num_entailles"].sum() / df["2021_num_entailles"].sum()

ratio_entailles_B_2041 = is_not_propice_2041_B_df["2021_num_entailles"].sum() / df["2021_num_entailles"].sum()
ratio_entailles_B_2071 = is_not_propice_2071_B_df["2021_num_entailles"].sum() / df["2021_num_entailles"].sum()

ratio_exploitations_A_2041 = is_not_propice_2041_A_df["2021_num_exploitations"].sum() / df["2021_num_exploitations"].sum()
ratio_exploitations_A_2071 = is_not_propice_2071_A_df["2021_num_exploitations"].sum() / df["2021_num_exploitations"].sum()

ratio_exploitations_B_2041 = is_not_propice_2041_B_df["2021_num_exploitations"].sum() / df["2021_num_exploitations"].sum()
ratio_exploitations_B_2071 = is_not_propice_2071_B_df["2021_num_exploitations"].sum() / df["2021_num_exploitations"].sum()


print(f"Scénario A (entailles): {round(ratio_entailles_A_2041, 3)} (2041), {round(ratio_entailles_A_2071, 3)} (2011)")
print(f"Scénario A (exploitations): {round(ratio_exploitations_A_2041, 3)} (2041), {round(ratio_exploitations_A_2071, 3)} (2011)")
print(f"Scénario B (entailles): {round(ratio_entailles_B_2041, 3)} (2041), {round(ratio_entailles_B_2071, 3)} (2071)")
print(f"Scénario B (exploitations): {round(ratio_exploitations_B_2041, 3)} (2041), {round(ratio_exploitations_B_2071, 3)} (2011)")

### À quel point les régions qui ont connu la plus forte croissance resteront-elles dans des zones propices au développement de l’érable ou se retrouveront-elles au contraire dans les zones où sa croissance pourrait être affectée/menacée?

On affiche les données ordonnées selon la perte d'entailles entre 2021 et 2011 (`diff_num_entailles`). On indique également si les régions se retrouvent dans la zone propice (`YYYY_X_is_propice`) selon l'année de début de la projection (`YYYY`) et le scénario (`X`). 

<span style="color:red"> 
Le climat des sept régions ayant connus la plus forte croissance du nombre d'entailles de 2011 à 2021 ne sera plus propice au développement de l'érable à sucre en 2071. 
</span>

In [ ]:
df.sort_values("diff_num_entailles", ascending=False).reset_index()[[
    "area_name",
    "province",
    "diff_num_entailles",
    "2041_A_is_propice",
    "2071_A_is_propice",
    "2041_B_is_propice",
    "2071_B_is_propice"
]].head(n=TOP_N)

In [ ]:
# TODO
# Proportion de la biomasse actuelle dans des zones qui ne sont pas propices?
# Proportion des entailles / exploitations dans des zones qui ne sont pas propices?
# Convertir num pixels en km carré.
# % Entailles par provinces. 